In [1]:
import os
import pandas as pd
import numpy as np
import json
import re
import duckdb

In [3]:
# Create DuckDB connection
con = duckdb.connect()

In [ ]:
# Loading reviews data
con.sql("CREATE TABLE reviews AS SELECT * FROM read_json_auto('../../data/yelp_raw_data/yelp_academic_dataset_review.json')")

reviews_df = con.sql("SELECT * FROM reviews").fetchdf()

In [ ]:
# Loading users data
con.sql("CREATE TABLE users AS SELECT * FROM read_json_auto('../../data/yelp_raw_data/yelp_academic_dataset_user.json')")

users_df = con.sql("SELECT * FROM users").fetchdf()

In [ ]:
# Loading business data
con.sql("CREATE TABLE business AS SELECT * FROM read_json_auto('../../data/yelp_raw_data/yelp_academic_dataset_business.json')")

business_df = con.sql("SELECT * FROM business").fetchdf()

In [7]:
reviews_df.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3.0,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5.0,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3.0,0,0,0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5.0,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03
4,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4.0,1,0,1,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15


In [8]:
reviews_df.shape

(6990280, 9)

In [9]:
users_df.head()

,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,qVc8ODYU5SZjKXVBgXdI7w,Walker,585,2007-01-25 16:47:26,7217,1259,5994,2007,"NSCy54eWehBJyZdG2iE84w, pe42u7DcCH2QmI81NX-8qA...",267,...,65,55,56,18,232,844,467,467,239,180
1,j14WgRoU_-2ZE1aw1dXrJg,Daniel,4333,2009-01-25 04:35:42,43091,13066,27281,"2009,2010,2011,2012,2013,2014,2015,2016,2017,2...","ueRPE0CX75ePGMqOFVj6IQ, 52oH4DrRvzzl8wh5UXyU0A...",3138,...,264,184,157,251,1847,7054,3131,3131,1521,1946
2,2WnXYQFK0hXEoTxPtV2zvg,Steph,665,2008-07-25 10:41:00,2086,1010,1003,"2009,2010,2011,2012,2013","LuO3Bn4f3rlhyHIaNfTlnA, j9B4XdHUhDfTKVecyWQgyA...",52,...,13,10,17,3,66,96,119,119,35,18
3,SZDeASXq7o05mMNLshsdIA,Gwen,224,2005-11-29 04:38:33,512,330,299,"2009,2010,2011","enx1vVPnfdNUdPho6PH_wg, 4wOcvMLtU6a9Lslggq74Vg...",28,...,4,1,6,2,12,16,26,26,10,9
4,hA5lMy-EnncsH4JoR-hFGQ,Karen,79,2007-01-05 19:40:59,29,15,7,,"PBK4q9KEEBHhFvSXCUirIw, 3FWPpM7KU1gXeOM_ZbYMbA...",1,...,1,0,0,0,1,1,0,0,0,0


In [10]:
users_df.shape

(1987897, 22)

In [11]:
business_df.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",NaN
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Monday': None, 'Tuesday': None, 'Wednesday':..."


In [12]:
business_df.shape

(150346, 14)

In [13]:
business_df.dtypes

business_id      object
name             object
address          object
city             object
state            object
postal_code      object
latitude        float64
longitude       float64
stars           float64
review_count      int64
is_open           int64
attributes       object
categories       object
hours            object
dtype: object

In [14]:
business_df['state'].value_counts()

state
PA     34039
FL     26330
TN     12056
IN     11247
MO     10913
LA      9924
AZ      9912
NJ      8536
NV      7715
AB      5573
CA      5203
ID      4467
DE      2265
IL      2145
TX         4
CO         3
WA         2
HI         2
MA         2
NC         1
UT         1
MT         1
MI         1
SD         1
XMS        1
VI         1
VT         1
Name: count, dtype: int64

In [15]:
business_df_pa = business_df[business_df['state'] == 'PA']

In [16]:
business_df_pa['city'].value_counts()

city
Philadelphia       14567
West Chester         838
King of Prussia      560
Doylestown           539
Bensalem             454
                   ...  
King of Prussi         1
Drexel Hil             1
Mt. Airy               1
Stroudsburg            1
UPPER MORELAND         1
Name: count, Length: 434, dtype: int64

In [17]:
business_df_philly = business_df_pa[business_df_pa['city'].str.match(r'^(Phil|PHL)', case = False)]

In [18]:
business_df_philly['city'].value_counts()

city
Philadelphia                       14567
Philadephia                            7
Phila                                  6
PHILADELPHIA                           4
PHILA                                  3
philadelphia                           3
Philadelphia PA                        2
Philadelphila                          1
Philiidelphia                          1
Philly                                 1
Philadelphia (Northeast Philly)        1
Philadelphia                           1
Philiadelphia                          1
Name: count, dtype: int64

In [19]:
# Creating a cleaned, standardized name for Philadelphia
business_df_philly.loc[:, 'city_cleaned'] = 'Philadelphia'

C:\Users\edwar\AppData\Local\Temp\ipykernel_1932\2974249043.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  business_df_philly.loc[:, 'city_cleaned'] = 'Philadelphia'


In [20]:
# Filtering for only restaurants
business_df_philly = business_df_philly[business_df_philly['categories'].str.match(r'Restau', case = False) & business_df_philly['categories'].notna()]

In [21]:
## Grabbing the top 200 open restaurants with the most reviews
business_df_philly = business_df_philly[business_df_philly['is_open'] == 1].sort_values('review_count', ascending = False)

# Extracting first 200 unique business_ids where is_open = 1
unique_business_ids = business_df_philly['business_id'].unique()[:200]

business_df_philly = business_df_philly[business_df_philly['business_id'].isin(unique_business_ids)]

In [22]:
business_df_philly

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,city_cleaned
20078,j-qtdD55OLfSqfsWuQTDJg,Parc,227 S 18th St,Philadelphia,PA,19103,39.949172,-75.170727,4.0,2761,1,"{'OutdoorSeating': 'True', 'RestaurantsGoodFor...","Restaurants, French, Wine Bars, Nightlife, Ame...","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'...",Philadelphia
149325,0RuvlgTnKFbX3IK0ZOOocA,Green Eggs Café,212 S 13th St,Philadelphia,PA,19107,39.948123,-75.162463,4.0,2679,1,"{'BusinessParking': '{'garage': False, 'street...","Restaurants, American (New), Diners, Breakfast...","{'Monday': '9:0-15:0', 'Tuesday': '9:0-15:0', ...",Philadelphia
113141,IWHdx0NhDKADkGOgXgOFKQ,Tommy DiNic's,1136 Arch St,Philadelphia,PA,19147,39.953478,-75.159396,4.0,2221,1,"{'NoiseLevel': 'u'loud'', 'GoodForKids': 'True...","Restaurants, American (Traditional), Sandwiches","{'Monday': '9:0-16:0', 'Tuesday': '9:0-18:0', ...",Philadelphia
129366,nIAbuktMEzVjT4P9pG89rQ,Buddakan,325 Chestnut St,Philadelphia,PA,19106,39.948919,-75.147167,4.0,1561,1,"{'BusinessAcceptsCreditCards': 'True', 'GoodFo...","Restaurants, Asian Fusion, Chinese","{'Monday': '17:0-22:0', 'Tuesday': '17:0-22:0'...",Philadelphia
147708,2CDI713ATuxHfnB5b-sBdw,Vedge,1221 Locust St,Philadelphia,PA,19107,39.947931,-75.161428,4.5,1447,1,"{'Caters': 'False', 'WiFi': 'u'no'', 'GoodForK...","Restaurants, Vegetarian, Beer, Wine & Spirits,...","{'Monday': '0:0-0:0', 'Tuesday': '17:0-22:0', ...",Philadelphia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130333,GKohndn_sMk2nWCh1jkAfg,Terryin,358 Snyder Ave,Philadelphia,PA,19148,39.921937,-75.152843,4.0,159,1,"{'RestaurantsTakeOut': 'True', 'RestaurantsAtt...","Restaurants, Chinese, Sushi Bars, Asian Fusion...","{'Monday': None, 'Tuesday': '11:0-22:30', 'Wed...",Philadelphia
65222,oVwEPg-BADIFl2SlGma6jg,Burrata,1247 S 13th St,Philadelphia,PA,19147,39.933925,-75.165243,4.5,155,1,"{'HasTV': 'False', 'WiFi': ''no'', 'Caters': '...","Restaurants, Italian","{'Monday': '0:0-0:0', 'Tuesday': '17:0-21:30',...",Philadelphia
118892,Ktg3ahIxk0JIkJwXAqu2ew,Hikari Sushi,"1040 N American St, Ste 701",Philadelphia,PA,19123,39.967402,-75.141153,5.0,155,1,"{'HasTV': 'False', 'RestaurantsPriceRange2': '...","Restaurants, Sushi Bars, Japanese","{'Monday': '16:30-22:0', 'Tuesday': '16:30-22:...",Philadelphia
85553,RNgb844JJbo6vO1iUod8vA,LongHorn Steakhouse,2120 S Columbus Blvd,Philadelphia,PA,19148,39.919412,-75.141194,2.5,155,1,"{'RestaurantsPriceRange2': '2', 'RestaurantsTa...","Restaurants, Steakhouses, American (Traditiona...","{'Monday': '0:0-0:0', 'Tuesday': '11:0-22:0', ...",Philadelphia


In [45]:
# Merging business_df_philly, reviews_df, users_df
bus_rev_df_philly = pd.merge(business_df_philly[['business_id', 'name']], reviews_df.drop(columns = ['funny', 'cool']), how = 'left', on = 'business_id', suffixes = ('_business', '_review'))
bus_rev_users_df_philly = pd.merge(bus_rev_df_philly, users_df[['user_id', 'name', 'review_count', 'average_stars', 'useful', 'elite']].rename(columns={'name': 'user_name'}), how = 'left', on = 'user_id', suffixes = ('_business_review', '_user'))

In [46]:
# Renaming columns in business_reviews_users_df
bus_rev_users_df_philly = bus_rev_users_df_philly.rename(columns = {'name': 'restaurant_name', 'stars': 'user_rating', \
                                                                    'useful_business_review': 'useful_restaurant_review_count', \
                                                                    'text': 'review', 'date': 'review_date', \
                                                                    'review_count': 'user_review_count', \
                                                                    'average_stars': 'user_average_rating', 'elite': 'user_elite_years', \
                                                                    'useful_user': 'useful_user_review_count'})

In [65]:
bus_rev_users_df_philly

,business_id,restaurant_name,review_id,user_id,user_rating,useful_restaurant_review_count,review,review_date,user_name,user_review_count,user_average_rating,useful_user_review_count,user_elite_years
0,j-qtdD55OLfSqfsWuQTDJg,Parc,bZe5yZntFLP2cDd1bAL9Fw,mzXDuhZBPdHfMIk1YXP0mQ,3.0,3,There is absolutely nothing wrong with this re...,2013-12-29 00:00:06,Kendrah,455,3.64,1512,
1,j-qtdD55OLfSqfsWuQTDJg,Parc,gww9NhRe3rhSu_HZGld7Pw,KuK8kZl4UQFNMbsn_FpXCA,3.0,0,great place for brunch! portions on certain d...,2012-04-28 14:14:57,Heather,97,4.01,28,
2,j-qtdD55OLfSqfsWuQTDJg,Parc,1HjdioSIL_mKUQf0ji3t_g,3_4Y3BFXlhFiCiCJZTUV2g,2.0,1,"My fiance loves French food, so for her birthd...",2018-05-28 22:06:24,Benjamin,33,4.42,18,
3,j-qtdD55OLfSqfsWuQTDJg,Parc,3iu_1SrlZNvtOEU3B9R73g,9kTsn8XJq3dcB7CMXkD0Xg,5.0,0,"Delicious food, great ambiance, excellent serv...",2016-10-26 18:24:06,Laurie,21,3.00,12,
4,j-qtdD55OLfSqfsWuQTDJg,Parc,FcyiG-L3btEbr0AOx67g4g,RNWJx8g-TIMVn1fpNFOCvA,2.0,0,I met a friend here for lunch and was disappoi...,2012-08-15 01:40:04,Sara,229,4.35,445,"2012,2013,2014,2015,2016"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
90386,CpV8X8DHk2VqWo11CEde-w,Paprica Modern Mediterranean Grill,rMv1q0Lrt7j2btp2iVnpug,td59mx4jPGvSrTChGgvFlw,4.0,6,"The food was mediocre, coming from a middle ea...",2019-11-06 02:37:53,Hana,83,4.07,187,"2018,2019,20,20,2021"
90387,CpV8X8DHk2VqWo11CEde-w,Paprica Modern Mediterranean Grill,t_eIWOqDE4tn71ZKRumD1Q,XJsVEc-ASmcugdoZoh3wyA,5.0,2,If you're looking Turkish spot in the city Pap...,2019-04-29 23:54:32,Asmaa,36,4.17,68,"2019,20,20,2021"
90388,CpV8X8DHk2VqWo11CEde-w,Paprica Modern Mediterranean Grill,kfxPBDNx9VAQdqaDXPyRfw,p-3GcxiOW6NDvrFq3Z0HVQ,5.0,1,Great authentic Turkish food. I TRUED PAPRİCA ...,2019-03-25 15:07:34,Kevin,91,4.52,107,"2017,2018,2019,20,20,2021"
90389,CpV8X8DHk2VqWo11CEde-w,Paprica Modern Mediterranean Grill,0gICsxvtkgE1_7bzIReSMA,sJGoA7bVO4Ef-HZtj5dayg,4.0,0,Delicious Turkish dining. The chicken shish k...,2021-09-18 00:57:45,Jeff,3,4.33,0,


In [48]:
business_df_philly.columns

Index(['business_id', 'name', 'address', 'city', 'state', 'postal_code',
       'latitude', 'longitude', 'stars', 'review_count', 'is_open',
       'attributes', 'categories', 'hours', 'city_cleaned'],
      dtype='object')

In [49]:
# Renaming columns in business_df and replacing city column with city_cleaned column and resetting_index
business_df_philly_1 = business_df_philly.drop(columns = 'city').rename(columns = {'name': 'restaurant_name', 'stars': 'restaurant_rating', \
                                                                                     'review_count': 'restaurant_review_count', 'city_cleaned': 'city'}) \
[['business_id', 'restaurant_name', 'address', 'city', 'state', 'postal_code', 'latitude', 'longitude', 'restaurant_review_count', 'is_open', \
  'attributes', 'categories', 'hours']].reset_index(drop = True)


In [50]:
business_df_philly_1

,business_id,restaurant_name,address,city,state,postal_code,latitude,longitude,restaurant_review_count,is_open,attributes,categories,hours
0,j-qtdD55OLfSqfsWuQTDJg,Parc,227 S 18th St,Philadelphia,PA,19103,39.949172,-75.170727,2761,1,"{'OutdoorSeating': 'True', 'RestaurantsGoodFor...","Restaurants, French, Wine Bars, Nightlife, Ame...","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'..."
1,0RuvlgTnKFbX3IK0ZOOocA,Green Eggs Café,212 S 13th St,Philadelphia,PA,19107,39.948123,-75.162463,2679,1,"{'BusinessParking': '{'garage': False, 'street...","Restaurants, American (New), Diners, Breakfast...","{'Monday': '9:0-15:0', 'Tuesday': '9:0-15:0', ..."
2,IWHdx0NhDKADkGOgXgOFKQ,Tommy DiNic's,1136 Arch St,Philadelphia,PA,19147,39.953478,-75.159396,2221,1,"{'NoiseLevel': 'u'loud'', 'GoodForKids': 'True...","Restaurants, American (Traditional), Sandwiches","{'Monday': '9:0-16:0', 'Tuesday': '9:0-18:0', ..."
3,nIAbuktMEzVjT4P9pG89rQ,Buddakan,325 Chestnut St,Philadelphia,PA,19106,39.948919,-75.147167,1561,1,"{'BusinessAcceptsCreditCards': 'True', 'GoodFo...","Restaurants, Asian Fusion, Chinese","{'Monday': '17:0-22:0', 'Tuesday': '17:0-22:0'..."
4,2CDI713ATuxHfnB5b-sBdw,Vedge,1221 Locust St,Philadelphia,PA,19107,39.947931,-75.161428,1447,1,"{'Caters': 'False', 'WiFi': 'u'no'', 'GoodForK...","Restaurants, Vegetarian, Beer, Wine & Spirits,...","{'Monday': '0:0-0:0', 'Tuesday': '17:0-22:0', ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,GKohndn_sMk2nWCh1jkAfg,Terryin,358 Snyder Ave,Philadelphia,PA,19148,39.921937,-75.152843,159,1,"{'RestaurantsTakeOut': 'True', 'RestaurantsAtt...","Restaurants, Chinese, Sushi Bars, Asian Fusion...","{'Monday': None, 'Tuesday': '11:0-22:30', 'Wed..."
196,oVwEPg-BADIFl2SlGma6jg,Burrata,1247 S 13th St,Philadelphia,PA,19147,39.933925,-75.165243,155,1,"{'HasTV': 'False', 'WiFi': ''no'', 'Caters': '...","Restaurants, Italian","{'Monday': '0:0-0:0', 'Tuesday': '17:0-21:30',..."
197,Ktg3ahIxk0JIkJwXAqu2ew,Hikari Sushi,"1040 N American St, Ste 701",Philadelphia,PA,19123,39.967402,-75.141153,155,1,"{'HasTV': 'False', 'RestaurantsPriceRange2': '...","Restaurants, Sushi Bars, Japanese","{'Monday': '16:30-22:0', 'Tuesday': '16:30-22:..."
198,RNgb844JJbo6vO1iUod8vA,LongHorn Steakhouse,2120 S Columbus Blvd,Philadelphia,PA,19148,39.919412,-75.141194,155,1,"{'RestaurantsPriceRange2': '2', 'RestaurantsTa...","Restaurants, Steakhouses, American (Traditiona...","{'Monday': '0:0-0:0', 'Tuesday': '11:0-22:0', ..."


In [51]:
# Unnesting the attributes column
business_df_philly_1_attr = pd.json_normalize(business_df_philly_1['attributes'])

In [52]:
business_df_philly_1_attr.columns

Index(['OutdoorSeating', 'RestaurantsGoodForGroups', 'RestaurantsReservations',
       'WiFi', 'BusinessAcceptsCreditCards', 'RestaurantsAttire',
       'RestaurantsPriceRange2', 'Alcohol', 'NoiseLevel', 'Caters', 'Ambience',
       'BusinessParking', 'BikeParking', 'HasTV', 'RestaurantsTableService',
       'DogsAllowed', 'RestaurantsDelivery', 'ByAppointmentOnly',
       'GoodForKids', 'RestaurantsTakeOut', 'HappyHour', 'GoodForMeal',
       'Music', 'Corkage', 'BYOBCorkage', 'BYOB', 'RestaurantsCounterService',
       'WheelchairAccessible', 'BusinessAcceptsBitcoin', 'DriveThru',
       'Smoking', 'BestNights', 'CoatCheck', 'GoodForDancing', 'AgesAllowed',
       'Open24Hours'],
      dtype='object')

In [53]:
# Counting the number of NA values in each column
business_df_philly_1_attr_na = business_df_philly_1_attr.isna().sum().sort_values()
business_df_philly_1_attr_na

BusinessAcceptsCreditCards      0
RestaurantsPriceRange2          0
RestaurantsDelivery             0
RestaurantsTakeOut              1
BusinessParking                 1
OutdoorSeating                  2
WiFi                            2
BikeParking                     2
RestaurantsGoodForGroups        3
RestaurantsReservations         3
NoiseLevel                      3
Caters                          3
HasTV                           4
Alcohol                         4
Ambience                        4
GoodForKids                     5
RestaurantsAttire               5
GoodForMeal                     8
RestaurantsTableService        71
HappyHour                      76
DogsAllowed                    84
WheelchairAccessible          118
BusinessAcceptsBitcoin        150
BYOBCorkage                   153
Music                         157
BestNights                    160
Corkage                       160
ByAppointmentOnly             164
GoodForDancing                165
BYOB          

In [54]:
# Grabbing columns with less than 10 NA values
business_df_philly_1_attr = business_df_philly_1_attr[business_df_philly_1_attr_na[business_df_philly_1_attr_na < 10].index]

In [55]:
business_df_philly_1_attr

,BusinessAcceptsCreditCards,RestaurantsPriceRange2,RestaurantsDelivery,RestaurantsTakeOut,BusinessParking,OutdoorSeating,WiFi,BikeParking,RestaurantsGoodForGroups,RestaurantsReservations,NoiseLevel,Caters,HasTV,Alcohol,Ambience,GoodForKids,RestaurantsAttire,GoodForMeal
0,True,3,True,True,"{'garage': False, 'street': True, 'validated':...",True,u'no',True,True,True,u'average',False,False,u'full_bar',"{'romantic': False, 'intimate': False, 'touris...",True,u'casual',"{'dessert': None, 'latenight': False, 'lunch':..."
1,False,2,True,True,"{'garage': False, 'street': True, 'validated':...",True,u'no',True,True,False,u'average',True,True,u'none',"{'touristy': False, 'hipster': False, 'romanti...",True,u'casual',"{'dessert': False, 'latenight': False, 'lunch'..."
2,True,2,True,True,"{'garage': True, 'street': True, 'validated': ...",False,u'no',True,False,False,u'loud',False,True,'none',"{'touristy': None, 'hipster': False, 'romantic...",True,'casual',"{'dessert': False, 'latenight': False, 'lunch'..."
3,True,3,True,True,"{'garage': False, 'street': True, 'validated':...",True,u'no',True,True,True,u'loud',False,False,'full_bar',"{'touristy': False, 'hipster': False, 'romanti...",False,'dressy',"{'dessert': None, 'latenight': False, 'lunch':..."
4,True,3,True,True,"{'garage': True, 'street': True, 'validated': ...",True,u'no',False,True,True,u'average',False,False,u'full_bar',"{'romantic': False, 'intimate': False, 'classy...",False,'casual',"{'dessert': None, 'latenight': False, 'lunch':..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,True,2,True,True,"{'garage': False, 'street': True, 'validated':...",False,u'no',True,True,False,u'quiet',True,True,u'none',"{u'divey': None, u'hipster': False, u'casual':...",True,u'casual',"{'dessert': False, 'latenight': False, 'lunch'..."
196,True,2,False,True,"{'garage': False, 'street': True, 'validated':...",True,'no',True,False,True,'average',True,False,u'none',"{'touristy': False, 'hipster': False, 'romanti...",False,'casual',"{'dessert': True, 'latenight': None, 'lunch': ..."
197,True,2,True,True,None,True,u'no',False,True,True,u'average',True,False,u'none',"{u'divey': False, u'hipster': False, u'casual'...",True,u'casual',"{'dessert': None, 'latenight': None, 'lunch': ..."
198,True,2,False,True,"{'garage': False, 'street': False, 'validated'...",False,u'free',True,True,False,u'average',False,True,u'full_bar',"{'romantic': False, 'intimate': False, 'classy...",True,'casual',"{'dessert': False, 'latenight': False, 'lunch'..."


In [56]:
# Unnesting the hours column
business_df_philly_1_hours = pd.json_normalize(business_df_philly_1['hours'])

In [57]:
# Creating columns representing each day of week checking to see if a restaurant is open that day of the week
def is_open(hours):
    if hours == '0:0-0:0' or hours == None:
        return 0
    else:
        return 1

for day_of_week in business_df_philly_1_hours.columns:
    business_df_philly_1_hours[f'{day_of_week}_is_open'] = business_df_philly_1_hours[day_of_week].apply(is_open)

business_df_philly_1_hours = business_df_philly_1_hours.drop(columns = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])

In [58]:
business_df_philly_1_hours

,Monday_is_open,Tuesday_is_open,Wednesday_is_open,Thursday_is_open,Friday_is_open,Saturday_is_open,Sunday_is_open
0,1,1,1,1,1,1,1
1,1,1,1,1,1,1,1
2,1,1,1,1,1,1,1
3,1,1,1,1,1,1,1
4,0,1,1,1,1,1,0
...,...,...,...,...,...,...,...
195,0,1,1,1,1,1,1
196,0,1,1,1,1,1,1
197,1,1,1,1,1,1,1
198,0,1,1,1,1,1,1


In [59]:
# merging business_df_philly_1, business_df_philly_1_attr, business_df_philly_1_hours
business_df_philly_fin = pd.concat([business_df_philly_1, business_df_philly_1_attr], axis = 1).drop(columns = 'attributes')
business_df_philly_fin = pd.concat([business_df_philly_fin, business_df_philly_1_hours], axis = 1).drop(columns = 'hours')

#merging back in hours columns at the end
business_df_philly_fin = pd.concat([business_df_philly_fin, business_df_philly_1['hours']], axis = 1)

In [60]:
business_df_philly_fin

,business_id,restaurant_name,address,city,state,postal_code,latitude,longitude,restaurant_review_count,is_open,...,RestaurantsAttire,GoodForMeal,Monday_is_open,Tuesday_is_open,Wednesday_is_open,Thursday_is_open,Friday_is_open,Saturday_is_open,Sunday_is_open,hours
0,j-qtdD55OLfSqfsWuQTDJg,Parc,227 S 18th St,Philadelphia,PA,19103,39.949172,-75.170727,2761,1,...,u'casual',"{'dessert': None, 'latenight': False, 'lunch':...",1,1,1,1,1,1,1,"{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'..."
1,0RuvlgTnKFbX3IK0ZOOocA,Green Eggs Café,212 S 13th St,Philadelphia,PA,19107,39.948123,-75.162463,2679,1,...,u'casual',"{'dessert': False, 'latenight': False, 'lunch'...",1,1,1,1,1,1,1,"{'Monday': '9:0-15:0', 'Tuesday': '9:0-15:0', ..."
2,IWHdx0NhDKADkGOgXgOFKQ,Tommy DiNic's,1136 Arch St,Philadelphia,PA,19147,39.953478,-75.159396,2221,1,...,'casual',"{'dessert': False, 'latenight': False, 'lunch'...",1,1,1,1,1,1,1,"{'Monday': '9:0-16:0', 'Tuesday': '9:0-18:0', ..."
3,nIAbuktMEzVjT4P9pG89rQ,Buddakan,325 Chestnut St,Philadelphia,PA,19106,39.948919,-75.147167,1561,1,...,'dressy',"{'dessert': None, 'latenight': False, 'lunch':...",1,1,1,1,1,1,1,"{'Monday': '17:0-22:0', 'Tuesday': '17:0-22:0'..."
4,2CDI713ATuxHfnB5b-sBdw,Vedge,1221 Locust St,Philadelphia,PA,19107,39.947931,-75.161428,1447,1,...,'casual',"{'dessert': None, 'latenight': False, 'lunch':...",0,1,1,1,1,1,0,"{'Monday': '0:0-0:0', 'Tuesday': '17:0-22:0', ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,GKohndn_sMk2nWCh1jkAfg,Terryin,358 Snyder Ave,Philadelphia,PA,19148,39.921937,-75.152843,159,1,...,u'casual',"{'dessert': False, 'latenight': False, 'lunch'...",0,1,1,1,1,1,1,"{'Monday': None, 'Tuesday': '11:0-22:30', 'Wed..."
196,oVwEPg-BADIFl2SlGma6jg,Burrata,1247 S 13th St,Philadelphia,PA,19147,39.933925,-75.165243,155,1,...,'casual',"{'dessert': True, 'latenight': None, 'lunch': ...",0,1,1,1,1,1,1,"{'Monday': '0:0-0:0', 'Tuesday': '17:0-21:30',..."
197,Ktg3ahIxk0JIkJwXAqu2ew,Hikari Sushi,"1040 N American St, Ste 701",Philadelphia,PA,19123,39.967402,-75.141153,155,1,...,u'casual',"{'dessert': None, 'latenight': None, 'lunch': ...",1,1,1,1,1,1,1,"{'Monday': '16:30-22:0', 'Tuesday': '16:30-22:..."
198,RNgb844JJbo6vO1iUod8vA,LongHorn Steakhouse,2120 S Columbus Blvd,Philadelphia,PA,19148,39.919412,-75.141194,155,1,...,'casual',"{'dessert': False, 'latenight': False, 'lunch'...",0,1,1,1,1,1,1,"{'Monday': '0:0-0:0', 'Tuesday': '11:0-22:0', ..."


In [61]:
# Counting number of unique restaurants
print(bus_rev_users_df_philly['business_id'].nunique())
print(bus_rev_users_df_philly['restaurant_name'].nunique())

200
198


In [ ]:
# Writing out bus_rev_users_df_philly to a csv file
bus_rev_users_df_philly.to_csv('../../data/data_cleaning/yelp_reviews_users_Phila.csv', index = False)

In [ ]:
# Writing out business_df_philly_fin to a csv file
business_df_philly_fin.to_csv('../../data/data_cleaning/yelp_restaurants_Phila.csv', index = False)